In [ ]:
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
import spacy
import requests

In [91]:
import pandas as pd
data = {
    "Age": [25],
    "Sex": [0],
    "Location": [1],
    "Product_1": [3258],
    "activity_at_injury": ["fall walk"],
    "object_involved": ["bike"],
    "injury_mechanism": [" "],
    "More details": ["fall from bike when riding"]
}

sample_df_single_record = pd.DataFrame(data)
sample_df_single_record

,Age,Sex,Location,Product_1,activity_at_injury,object_involved,injury_mechanism,More details
0,25,0,1,3258,fall walk,bike,,fall from bike when riding


In [92]:
bdpt_dict={}
bdpt_dict[0]='INTERNAL'
bdpt_dict[30]='SHOULDER'
bdpt_dict[31]='UPPERTRUNK'
bdpt_dict[32]='ELBOW'
bdpt_dict[33]='LOWERARM'
bdpt_dict[34]='WRIST'
bdpt_dict[35]='KNEE'
bdpt_dict[36]='LOWERLEG'
bdpt_dict[37]='ANKLE'
bdpt_dict[38]='PUBICREGION'
bdpt_dict[75]='HEAD'
bdpt_dict[76]='FACE'
bdpt_dict[77]='EYEBALL'
bdpt_dict[78]='UPPERTRUNK(OLD)'
bdpt_dict[79]='LOWERTRUNK'
bdpt_dict[80]='UPPERARM'
bdpt_dict[81]='UPPERLEG'
bdpt_dict[82]='HAND'
bdpt_dict[83]='FOOT'
bdpt_dict[84]='25-50% OF BODY'
bdpt_dict[85]='ALLPARTSBODY'
bdpt_dict[86]='OTHER(OLD)'
bdpt_dict[87]='NOTSTATED/UNK'
bdpt_dict[88]='MOUTH'
bdpt_dict[89]='NECK'
bdpt_dict[90]='LOWERARM(OLD)'
bdpt_dict[91]='LOWERLEG(OLD)'
bdpt_dict[92]='FINGER'
bdpt_dict[93]='TOE'
bdpt_dict[94]='EAR'

In [93]:
sample_df_single_record['Narrative_LLM']=sample_df_single_record["activity_at_injury"].astype(str) + ' '+sample_df_single_record["injury_mechanism"].astype(str)+ ' ' + sample_df_single_record["object_involved"].astype(str)
sample_df_single_record['Narrative']=sample_df_single_record["More details"]
sample_df_single_record_2=sample_df_single_record[["Age","Sex","Location","Product_1"]]
sample_df_single_record_2

,Age,Sex,Location,Product_1
0,25,0,1,3258


In [95]:
# Load the SpaCy model
nlp = spacy.load('en_core_web_md')

sematic_distance=[]

for key in bdpt_dict:

    #print(key)
    #print(i)
    word1 = bdpt_dict[key] #"cut"
    word2 = sample_df_single_record.at[0,'Narrative_LLM'] #"bike"

    # SpaCy similarity
    doc1 = nlp(word1)
    doc2 = nlp(word2)
    similarity = doc1.similarity(doc2)
    sematic_distance.append(similarity)

0
30
31
32
33
34
35
36
37
38
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94


/tmp/ipykernel_7972/2849378550.py:19: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = doc1.similarity(doc2)


In [96]:
corpus = sample_df_single_record['Narrative'].fillna('')

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    tagged_tokens = nltk.pos_tag(tokens)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, pos='v') for token, pos in tagged_tokens]
    return ' '.join(lemmatized_tokens)

sample_df_single_record['Processed_Narrative'] = corpus.apply(preprocess_text)

with open('tfidf_vectorizer.pkl', 'rb') as file:
    loaded_vectorizer = pickle.load(file)

# Use the loaded vectorizer to transform new data
tfidf_matrix = loaded_vectorizer.transform(sample_df_single_record['Processed_Narrative'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=loaded_vectorizer.get_feature_names_out())

In [97]:
corpus_LLM = sample_df_single_record['Narrative_LLM'].fillna('')

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    tagged_tokens = nltk.pos_tag(tokens)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, pos='v') for token, pos in tagged_tokens]
    return ' '.join(lemmatized_tokens)

sample_df_single_record['Processed_Narrative_LLM'] = corpus_LLM.apply(preprocess_text)

# Create a TfidfVectorizer object
with open('tfidf_vectorizer_LLM.pkl', 'rb') as file:
    loaded_vectorizer_llm = pickle.load(file)

# Use the loaded vectorizer to transform new data
tfidf_matrix_LLM = loaded_vectorizer_llm.transform(sample_df_single_record['Processed_Narrative_LLM'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df_LLM = pd.DataFrame(tfidf_matrix_LLM.toarray(), columns=loaded_vectorizer_llm.get_feature_names_out())
tfidf_df_LLM=tfidf_df_LLM.add_suffix('_LLM')

# Concatenate the TF-IDF features with your existing data

#data_ready = pd.concat([data_core_sample, tfidf_df_LLM], axis=1)

In [98]:
data = {
    "Body_Part": [25],
}
df_body= pd.DataFrame(data)
data_ready = pd.concat([sample_df_single_record_2[['Age','Sex','Location']],df_body,sample_df_single_record_2[['Product_1']], tfidf_df,tfidf_df_LLM], axis=1)

In [109]:
with open('X_scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

prob_out={}

with open('xgboost_tfidf_LLM_v2.pkl', 'rb') as file:
    best_model = pickle.load(file)

k=0

for i in bdpt_dict:
    X_test_final.at[0,'Body_Part']=i
    #print(X_test_final)
    temp=pd.DataFrame()
    temp['sematic_distance']=[sematic_distance[k]]
    #print(temp)
    data_ready_2=pd.concat([temp,data_ready], axis=1)
    #print(data_ready_2)

    X_test_scaled = scaler.transform(data_ready_2.select_dtypes(include=['number']))
    X_test_final=pd.DataFrame(X_test_scaled, columns=data_ready_2.select_dtypes(include=['number']).columns)
    y_prob = best_model.predict_proba(X_test_final)[:, 1][0]
    #print(y_prob)
    prob_out[str(i)]=y_prob
    k+=1

prob_out

{'0': 0.1570866,
 '30': 0.1570866,
 '31': 0.39655313,
 '32': 0.1570866,
 '33': 0.39655313,
 '34': 0.1570866,
 '35': 0.1570866,
 '36': 0.39655313,
 '37': 0.1570866,
 '38': 0.39655313,
 '75': 0.1570866,
 '76': 0.1570866,
 '77': 0.39655313,
 '78': 0.19196437,
 '79': 0.39655313,
 '80': 0.39655313,
 '81': 0.39655313,
 '82': 0.1570866,
 '83': 0.1570866,
 '84': 0.1570866,
 '85': 0.39655313,
 '86': 0.19196437,
 '87': 0.21336558,
 '88': 0.1570866,
 '89': 0.1570866,
 '90': 0.19196437,
 '91': 0.19196437,
 '92': 0.1570866,
 '93': 0.1570866,
 '94': 0.1570866}

In [102]:
xxxx
def Predict_Body_parts_Prob(Age, Sex, Race,Location,Hispanic, Product_1 ,Alcohol, Drug):
    X = np.column_stack([Age, Sex, Race,Location,Hispanic, Product_1 ,Alcohol, Drug])
    
    return loaded_model.predict_proba(X)

NameError: name 'data_ready_2' is not defined

In [87]:
!python -m spacy download en_core_web_md

/anaconda/envs/azureml_py38/lib/python3.10/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 30.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
